In [1]:
!pip install -q sklearn

In [2]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import files
uploaded = files.upload()

Saving UniversalBank.csv to UniversalBank.csv


In [5]:
import io
df = pd.read_csv(io.BytesIO(uploaded['UniversalBank.csv']))

In [6]:
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [7]:
df.drop(["ID"], axis="columns",inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 5000 non-null   int64  
 1   Experience          5000 non-null   int64  
 2   Income              5000 non-null   int64  
 3   ZIP Code            5000 non-null   int64  
 4   Family              5000 non-null   int64  
 5   CCAvg               5000 non-null   float64
 6   Education           5000 non-null   int64  
 7   Mortgage            5000 non-null   int64  
 8   Personal Loan       5000 non-null   int64  
 9   Securities Account  5000 non-null   int64  
 10  CD Account          5000 non-null   int64  
 11  Online              5000 non-null   int64  
 12  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 507.9 KB


In [9]:
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [10]:
#split to training and testing
X=df.drop("Age", axis="columns")
Y=df["Age"]

In [11]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.85, random_state=2)

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
lr = LogisticRegression()

In [16]:
#test the graph on logistic regression(also called linear classification)
crossval = cross_val_score(lr,X_train,Y_train,scoring='accuracy',cv=10).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,


In [17]:
crossval

0.19035294117647056

In [18]:
#fit classifier into training set
svc = SVC(kernel='linear')
svc.fit(X_train,Y_train)

SVC(kernel='linear')

In [19]:
# import metrics to compute accuracy
from sklearn.metrics import accuracy_score

In [20]:
Y_pred=svc.predict(X_test)
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(Y_test, Y_pred)))

Model accuracy score with default hyperparameters: 0.2920


In [21]:
#letsrun svm with fewer outliers
linear_svc=SVC(kernel='linear', C=100.0) 
linear_svc.fit(X_train,Y_train)
y_pred_test=linear_svc.predict(X_test)
print('Model accuracy score with linear kernel and C= : {0:0.4f}'. format(accuracy_score(Y_test, y_pred_test)))

Model accuracy score with linear kernel and C= : 0.3360


In [22]:
from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV
import multiprocessing

In [23]:
# we test grid search SVM
#kernel parameters using polynomial and sigmoid kernel
param_grid = [
 {
 'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
 'C': [ 0.1, 1.0, 10.0, 100.0]  
 }
]
gs= GridSearchCV(estimator=SVC(),param_grid=param_grid,scoring='accuracy',cv=8,n_jobs=multiprocessing.cpu_count())
gs.fit(X_train,Y_train)

GridSearchCV(cv=8, estimator=SVC(), n_jobs=2,
             param_grid=[{'C': [0.1, 1.0, 10.0, 100.0],
                          'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}],
             scoring='accuracy')

In [24]:
gs.best_estimator_

SVC(C=100.0, kernel='linear')